In [110]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

In [111]:
sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

mktdir = "./MkTdata"

# force=True read from alphavantage server
# force=False try to read first from the local directory if data exists if not from alphavantage
mktdata = az.readMkT(symb, dstart = sdate, dend = edate, 
                     dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


In [112]:
alpha_CVaR98 = [0.96]
alpha_CVaR97 = [0.95]
alpha_CVaR95 = [0.94]
alpha_CVaR = alpha_CVaR98 + alpha_CVaR97 + alpha_CVaR95

coef_MAD1 = [1.]
coef_MAD2 = [0.5] * 2
coef_MAD3 = [1./4] * 4

rtype = 'InvNrisk'
rtype = 'Sharpe'
#rtype = 'MinRisk'

mu = 0.0

hlength1 = 4

models = {'P_CVaR98': {'type': 'Port_CVaR', 'm_param': {'mu': mu, 'alpha': alpha_CVaR98, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR97': {'type': 'Port_CVaR', 'm_param': {'mu': mu, 'alpha': alpha_CVaR97, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR95': {'type': 'Port_CVaR', 'm_param': {'mu': mu, 'alpha': alpha_CVaR95, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'mu': mu, 'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD1': {'type': 'Port_MAD', 'm_param': {'mu': mu, 'coef': coef_MAD1, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD2': {'type': 'Port_MAD', 'm_param': {'mu': mu, 'coef': coef_MAD2, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD3': {'type': 'Port_MAD', 'm_param': {'mu': mu, 'coef': coef_MAD3, 'rtype': rtype, 'hlength': hlength1}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}


In [113]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

In [114]:
pdata = pd.concat(port, axis=1, join='inner').melt(var_name='symbol', value_name='close', ignore_index=False)

In [115]:
ps = az.Port_Simple(pdata, col='close', pname='ALL')
ports = ps.set_model()

In [116]:
_ = ps.port_view_all(sdate=pd.to_datetime('2000-01-01'), componly=True, fancy=True)

In [117]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,DD_date,DD_start,DD_end
symbol,,,,,
P_CVaR,16.84,-20.73,2020-03-20,2020-02-19,2020-05-07
P_CVaR95,17.03,-21.90,2020-03-20,2020-02-19,2020-05-08
P_CVaR97,16.99,-20.83,2020-03-20,2020-02-19,2020-05-07
P_CVaR98,16.76,-20.46,2020-03-20,2020-02-19,2020-05-07
P_MAD1,18.55,-27.08,2020-03-20,2020-02-19,2020-05-18
P_MAD2,18.75,-26.58,2020-03-20,2020-02-19,2020-05-11
P_MAD3,18.52,-26.11,2020-03-20,2020-02-19,2020-05-18
P_N,17.73,-17.43,2020-03-18,2020-02-19,2020-04-14


In [118]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_CVaR,P_CVaR95,P_CVaR97,P_CVaR98,P_MAD1,P_MAD2,P_MAD3,P_N
year,,,,,,,,
2016,0.36%,0.47%,0.53%,0.44%,2.80%,2.03%,1.34%,3.08%
2017,24.41%,24.89%,24.36%,24.35%,27.57%,27.83%,27.79%,21.57%
2018,5.97%,6.21%,5.60%,5.62%,6.59%,5.53%,5.21%,7.69%
2019,29.49%,30.09%,30.15%,30.01%,34.49%,35.37%,34.22%,25.06%
2020,27.58%,26.85%,28.25%,27.01%,27.44%,28.92%,29.33%,32.55%
2021,-1.06%,-0.85%,-1.15%,-1.04%,-1.70%,-1.32%,-1.12%,1.63%
